In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import sklearn as sk

sys.path.append('C:\\Users\\dalun\\PycharmProjects\\Thesiss')

import qtLibrary.libquanttree as qt
import main_code.neuralNetworks as nn
import pickle as pk
from main_code.auxiliary_project_functions import create_bins_combination


In [21]:
#Create dataframe from saved thresholds

frame_1N = pd.read_csv('File_N_and_thr_0_01')
frame_5N = pd.read_csv('File_N_and_thr_0_5')
frame_1A = pd.read_csv('Asymptotic_0._1')
frame_5A = pd.read_csv('Asymptotic_0_5')

frames = [frame_1N, frame_1A, frame_5N, frame_5A]

for frame in frames:
    frame = frame.iloc[:, 1:]

# frame_1N = frame_1N.iloc[:,1:]
# frame_1A = frame_1A.iloc[:,1:]
# frame_5N = frame_5N.iloc[:,1:]
# frame_5A = frame_5A.iloc[:,1:]
    

training_1N = frame_1N.iloc[:, np.r_[:frame.shape[1] - 1, -1]]
thresholds_1N = frame_1N.iloc[:,-2]

training_5N = frame_5N.iloc[:, np.r_[:frame.shape[1] - 1, -1]]
thresholds_5N = frame_5N.iloc[:,-2]

training_1A = frame_1A.iloc[:, 1:-1]
thresholds_1A = frame_1A.iloc[:,-1]

training_5A = frame_5A.iloc[:, 1:-1]
thresholds_5A = frame_5A.iloc[:,-1]
    
for threshold in [thresholds_1N, thresholds_5N, thresholds_1A, thresholds_5A]:
    print(np.var(threshold))




0.16947259348224802
0.06604053528243103
0.07034939431321958
0.08242142618464099


In [22]:
#Train networks

nodes = 10
print('nodes: ' + str(nodes))
easy_cor_1N = np.array(training_1N) #.iloc[:,-3:]
easy_cor_5N = np.array(training_5N) #.iloc[:,-3:]
easy_cor_1A = np.array(training_1A) #.iloc[:,-3:]
easy_cor_5A = np.array(training_5A) #.iloc[:,-3:]


easy_cor = [easy_cor_1N, easy_cor_5N, easy_cor_1A, easy_cor_5A]
thresholds = [thresholds_1N, thresholds_5N, thresholds_1A, thresholds_5N]

learner_1N = sk.neural_network.MLPRegressor(early_stopping=True, hidden_layer_sizes=nodes, solver='lbfgs', learning_rate='constant', max_iter=10000, n_iter_no_change=40, verbose=False, random_state=True, alpha = 0)
learner_5N = sk.neural_network.MLPRegressor(early_stopping=True, hidden_layer_sizes=nodes, solver='lbfgs', learning_rate='constant', max_iter=10000, n_iter_no_change=40, verbose=False, random_state=True, alpha = 0)
learner_1A = sk.neural_network.MLPRegressor(early_stopping=True, hidden_layer_sizes=nodes, solver='lbfgs', learning_rate='constant', max_iter=10000, n_iter_no_change=40, verbose=False, random_state=True, alpha = 0)
learner_5A = sk.neural_network.MLPRegressor(early_stopping=True, hidden_layer_sizes=nodes, solver='lbfgs', learning_rate='constant', max_iter=10000, n_iter_no_change=40, verbose=False, random_state=True, alpha = 0)

learners = [learner_1N, learner_5N, learner_1A, learner_5A]

for index in range(len(learners)):
    learners[index].fit(easy_cor[index], thresholds[index])
    print(learners[index].score(easy_cor[index], thresholds[index]))
    

nodes: 10
0.4621412471968213
0.18851229703268557
0.1147419766566825
0.0003459488000780997


In [23]:
#Test 
nu = 32
statistic = qt.tv_statistic
# print(easy_cor_1N[0])

tested = 10
alpha = [0.5]
values = np.zeros([tested, 3])
for index in range(tested):
    regressor = learner_5N
    histogram = create_bins_combination(8)
    tree = qt.QuantTree(histogram)
    number = np.random.randint(10, 1000)
    tree.ndata =number
    rich_histogram = np.append(histogram, float(number))
    # print(len(rich_histogram), easy_cor_1N.shape[1]) 
    threshold_computer = qt.ChangeDetectionTest(tree, nu, statistic)
    threshold = threshold_computer.estimate_quanttree_threshold(alpha, 3000)
    threshold2 = threshold_computer.estimate_quanttree_threshold(alpha, 3000)   
    # rich_histogram = scaler.transform(rich_histogram.reshape(1, -1))
    threshold3 = regressor.predict(rich_histogram.reshape(1, -1))
    values[index] = np.array([ threshold, threshold2, threshold3])
final_frame = pd.DataFrame(values)
final_frame['inner differences'] = np.abs(values[:,0] - values[:,1])
final_frame['differences'] = np.abs(values[:,0] - values[:,2])
final_frame =  final_frame.rename(columns={0:'trhesholds', 1 : 'thresholds2', 2:'regressions' })
print('alpha = '+str(alpha))
print(final_frame.describe())

tested = 10
alpha = [0.01]
values = np.zeros([tested, 3])
for index in range(tested):
    regressor = learner_1N
    histogram = create_bins_combination(8)
    tree = qt.QuantTree(histogram)
    number = np.random.randint(10, 1000)
    tree.ndata =number
    rich_histogram = np.append(histogram, float(number))
    # print(len(rich_histogram), easy_cor_1N.shape[1]) 
    threshold_computer = qt.ChangeDetectionTest(tree, nu, statistic)
    threshold = threshold_computer.estimate_quanttree_threshold(alpha, 3000)
    threshold2 = threshold_computer.estimate_quanttree_threshold(alpha, 3000)   
    # rich_histogram = scaler.transform(rich_histogram.reshape(1, -1))
    threshold3 = regressor.predict(rich_histogram.reshape(1, -1))
    values[index] = np.array([ threshold, threshold2, threshold3])
final_frame = pd.DataFrame(values)
final_frame['inner differences'] = np.abs(values[:,0] - values[:,1])
final_frame['differences'] = np.abs(values[:,0] - values[:,2])
final_frame =  final_frame.rename(columns={0:'trhesholds', 1 : 'thresholds2', 2:'regressions' })
print('alpha = '+str(alpha))
print(final_frame.describe())
#print ('r2 is ' + str(r2_score(final_frame['trhesholds'], final_frame['regressions'])))
#print('inner r2 is ' + str(r2_score(final_frame['trhesholds'], final_frame['thresholds2'])))

#Dictionary creation





alpha = [0.5]
       trhesholds  thresholds2  regressions  inner differences  differences
count   10.000000    10.000000    10.000000          10.000000    10.000000
mean     6.014363     6.030089     5.809328           0.041970     0.308812
std      0.437716     0.433220     0.034859           0.041254     0.342290
min      5.554043     5.630245     5.769475           0.000000     0.033280
25%      5.730636     5.749687     5.782469           0.014774     0.051293
50%      5.775417     5.791478     5.795172           0.032870     0.163243
75%      6.341100     6.296440     5.842483           0.056242     0.540895
max      6.786295     6.868905     5.858401           0.132836     0.952972
alpha = [0.01]
       trhesholds  thresholds2  regressions  inner differences  differences
count   10.000000    10.000000    10.000000          10.000000    10.000000
mean    10.151529    10.173365    10.323129           0.234276     0.273440
std      0.228670     0.180329     0.066293           0.184

## Pickle the trained net with the characteristics

In [24]:
#Pickle: store

dictionary = {('normal', 0.01): learner_1N, ('normal', 0.5): learner_5N,
              ('asymptotic', 0.01): learner_1A, ('Asymptotic', 0.5): learner_5A}


with open('network.pickle', 'wb') as handle:
    pk.dump(dictionary, handle)


In [25]:
#Pickle: verify it works

file2 = open('network.pickle', 'rb')
new_d = pk.load(file2)
print(new_d[('normal', 0.01)])
file2.close()
new_d[('normal', 0.01)].predict(rich_histogram.reshape(1, -1))

MLPRegressor(activation='relu', alpha=0, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=True, epsilon=1e-08,
             hidden_layer_sizes=10, learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=10000,
             momentum=0.9, n_iter_no_change=40, nesterovs_momentum=True,
             power_t=0.5, random_state=True, shuffle=True, solver='lbfgs',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)


array([10.35841553])